In [ ]:
#!pip -qq install gradio

In [1]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [8]:
import openai
import os
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from tqdm import tqdm
#import gradio

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "sk-proj-5BnnFEn2CHKp6HpRAQ1gT3BlbkFJXY97BE4C0ZyhxBQQdjeZ"

In [ ]:
#camera, battery, display,design,

#Extract Aspect,sentiment

In [9]:
def get_tag(sentence):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": """
            You are an aspect-based sentiment analysis tagger. Given a sentence, you need to return a JSON object of the following format:
            [
                {
                    "aspect": "<term>",
                    "sentiment_term": "<term>",
                    "sentiment_category": "<category>"
                }
            ]
            If there are multiple aspect-sentiment pairs, include multiple elements in the array.

            Guidelines:
            - Aspect term should be camera, battery, display,design for the product. It should not related to a concept or any human being
            - Avoid redundant aspects. Each aspect should be from one of these category
                    -- camera
                    -- battery
                    -- display
                    -- design
                    -- software
                    -- cpu/gpu
                    -- memory
                    -- network
            - Don't assume any aspecs .Only follow the above 8 aspects.Each aspect should be from one of these category
            - The aspect term should clearly represent a distinct part or feature of the product
            - The sentiment term should reflect the sentiment related to the aspect.
            - Ensure that categories are concise and relevant.
            - output should be a valid json and no other keywords

            Example:
            Input: "The picture quality is great but the phone discharged very quickly."
            Output: [
                {
                    "aspect": "camera",
                    "sentiment_term": "great",
                    "sentiment_category": "positive"
                },
                {
                    "aspect": "battery",
                    "sentiment_term": "discharged quickly",
                    "sentiment_category": "negative"
                }
            ],
            Input: "The screen quality is great but the design is terrible."
            Output: [
                {
                    "aspect": "display",
                    "sentiment_term": "great",
                    "sentiment_category": "positive"
                },
                {
                    "aspect": "design",
                    "sentiment_term": "terrible",
                    "sentiment_category": "negative"
                }
            ]
            """},
            {"role": "user", "content": sentence}
        ]
    )
    #return response.choices[0].text.strip()
    return response.choices[0].message.content


# Test on one example

In [10]:

def extract_aspects(review,row_id=1):
    json_output = get_tag(review)
    aspects_data = json.loads(json_output)
    aspects_df= pd.DataFrame(aspects_data)
    aspects_df['review']=review
    aspects_df['row_id']=row_id
    return aspects_df


In [11]:
extract_aspects('i love food')

,review,row_id


In [12]:
comment=('''I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Reception is not too shabby. I was using it in the elevator which is a remarkable feat considering my old phone would lose service by simply putting it in my pocket. ===> Compared to my old Samsung, this phone works quite well. The ring tones are loud enough to hear and the phone actually charges quickly and has great battery life. It doesn't heat up like a potatoe in the oven either during long phone convos. ===> Nice bright, large screen. ===> Cute ways to customize it. Scroll bar can be set to purple, pink, aqua, orange, etc. Overall: Okay phone. It serves its purpose but definitely pales in comparison
to these new phones coming out from Sprint. Why get so so when you can get great?''')

In [13]:
extract_aspects(comment)

,aspect,sentiment_term,sentiment_category,review,row_id
0,software,takes forever to get to what you want because ...,negative,I had the Samsung A600 for awhile which is abs...,1
1,software,no fast way to silence,negative,I had the Samsung A600 for awhile which is abs...,1
2,design,mission to get off and will break ur nails,negative,I had the Samsung A600 for awhile which is abs...,1
3,design,Buttons could be a bit bigger,negative,I had the Samsung A600 for awhile which is abs...,1
4,software,Vibration could be stronger,negative,I had the Samsung A600 for awhile which is abs...,1
5,network,Reception is not too shabby,positive,I had the Samsung A600 for awhile which is abs...,1
6,network,works quite well,positive,I had the Samsung A600 for awhile which is abs...,1
7,battery,charges quickly and has great battery life,positive,I had the Samsung A600 for awhile which is abs...,1
8,cpu/gpu,doesn't heat up like a potato in the oven,positive,I had the Samsung A600 for awhile which is abs...,1
9,display,"Nice bright, large screen",positive,I had the Samsung A600 for awhile which is abs...,1


#Read review data

In [14]:
df=pd.read_csv('/content/product_sample.csv')


In [15]:
df

,Unnamed: 0,asin,name,rating,date,verified,title_x,body,helpfulVotes,brand,product_title,image,totalReviews,price,originalPrice
0,8198,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.00,0.00
1,8325,B00F2SKPIM,Cynthia G Sercer,5,"May 5, 2015",True,Five Stars,emjoy this phone,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.00,0.00
2,8276,B00F2SKPIM,Doreen Demars,5,"August 20, 2015",True,Five Stars,Nice phone,1.0,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.00,0.00
3,8530,B00F2SKPIM,JCinPC,5,"April 1, 2014",True,terrific phone,"Everyone at work laughs at me when I ""answer m...",NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.00,0.00
4,8067,B00F2SKPIM,Nero,5,"March 7, 2017",True,Amazing looks like new,Love it work geart on Verizon. I am very pleased,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,34049,B06Y16RL4W,Shanice Robert,1,"September 23, 2019",True,Scam,Scam,NaN,Samsung,Samsung Galaxy S8+ 64GB Factory Unlocked Smart...,https://m.media-amazon.com/images/I/51wJeaY3ek...,793,399.98,589.99
96,33912,B06Y16RL4W,Amazon Customer,5,"January 21, 2019",True,Great product for a great price!,Great product for a great price!,NaN,Samsung,Samsung Galaxy S8+ 64GB Factory Unlocked Smart...,https://m.media-amazon.com/images/I/51wJeaY3ek...,793,399.98,589.99
97,34358,B06Y16RL4W,F. G. Fowler,5,"October 22, 2018",True,Great phone deal!,Everything was as expected! No issues with the...,NaN,Samsung,Samsung Galaxy S8+ 64GB Factory Unlocked Smart...,https://m.media-amazon.com/images/I/51wJeaY3ek...,793,399.98,589.99
98,34156,B06Y16RL4W,vanessa,1,"June 8, 2018",True,One Star,The phone was used not new like they advertise...,2.0,Samsung,Samsung Galaxy S8+ 64GB Factory Unlocked Smart...,https://m.media-amazon.com/images/I/51wJeaY3ek...,793,399.98,589.99


In [16]:
df['row_id'] = range(1, len(df) + 1)

In [18]:
df=df[['asin', 'name', 'rating', 'date', 'verified', 'title_x',
       'body', 'helpfulVotes', 'brand', 'product_title', 'image', 'totalReviews',
       'price', 'originalPrice', 'row_id']].copy()

In [19]:
df.columns

Index(['asin', 'name', 'rating', 'date', 'verified', 'title_x', 'body',
       'helpfulVotes', 'brand', 'product_title', 'image', 'totalReviews',
       'price', 'originalPrice', 'row_id'],
      dtype='object')

In [20]:
def process_data(df):
    arr = []
    for i,x in tqdm(df.iterrows()):
        try:
            review = x['body']
            row_id = x['row_id']
            result = get_tag(review)
            tags = json.loads(result)
            for t in tags:
                t['row_id'] = row_id
                arr.append(t)
        except:
            print(i)

    return pd.DataFrame(arr)

#$41.41

In [21]:
aspect_data=process_data(df)

6it [00:11,  1.66s/it]

5


37it [01:03,  1.93s/it]

36


50it [01:26,  1.38s/it]

49


85it [02:35,  1.14it/s]

84


100it [02:55,  1.75s/it]


In [22]:
aspect_data.head()

,aspect,sentiment_term,sentiment_category,row_id
0,design,fantastic,positive,1
1,cpu/gpu,crazy fast,positive,1
2,display,size,positive,1
3,design,terrific,positive,4
4,battery,Charge the battery,neutral,4


# Merging the aspect sentiment information with original electronics data

In [23]:
final_data=pd.merge(df,aspect_data,on='row_id',how='inner')

In [24]:
final_data.head()

,asin,name,rating,date,verified,title_x,body,helpfulVotes,brand,product_title,image,totalReviews,price,originalPrice,row_id,aspect,sentiment_term,sentiment_category
0,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,1,design,fantastic,positive
1,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,1,cpu/gpu,crazy fast,positive
2,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,1,display,size,positive
3,B00F2SKPIM,JCinPC,5,"April 1, 2014",True,terrific phone,"Everyone at work laughs at me when I ""answer m...",NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,4,design,terrific,positive
4,B00F2SKPIM,JCinPC,5,"April 1, 2014",True,terrific phone,"Everyone at work laughs at me when I ""answer m...",NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,4,battery,Charge the battery,neutral


In [25]:
#final_data.to_csv('final_aspect_sentiment.csv')

#Gradio for demonstartion

In [ ]:
def get_html_output(aspects_data):
    html_output = """
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-size: 18px;
            text-align: left;
        }
        table th, table td {
            padding: 12px;
            border: 1px solid #ddd;
        }
        table th {
            background-color: #f2f2f2;
        }
        table tr:nth-child(even) {
            background-color: #f9f9f9;
        }
        table tr:hover {
            background-color: #f1f1f1;
        }
        .positive {
            background-color: #d4edda;
        }
        .negative {
            background-color: #f8d7da;
        }
    </style>
    <table>
        <tr>
            <th>Aspect</th>
            <th>Aspect Category</th>
            <th>Sentiment</th>
            <th>Sentiment Category</th>
        </tr>"""

    for item in aspects_data:
        sentiment_class = "positive" if item['sentiment_category'] == "positive" else "negative" if item['sentiment_category'] == "negative" else ""
        html_output += f"<tr><td>{item['aspect_term']}</td><td>{item['aspect_category']}</td><td class='{sentiment_class}'>{item['sentiment_term']}</td><td class='{sentiment_class}'>{item['sentiment_category']}</td></tr>"

    html_output += "</table>"

    return html_output


In [ ]:
def aspect_extraction_gradio(review):
    json_output = get_tag(review)
    aspects_data = json.loads(json_output)
    return get_html_output(aspects_data)


In [ ]:
in_review = gradio.Textbox(lines=2, placeholder=None, value="review", label='Enter Review Text')

In [ ]:
#out_label = gradio.Textbox(type="text", label='Aspect_Label')

In [ ]:
iface = gradio.Interface(fn = aspect_extraction_gradio,
                         inputs = in_review,
                         outputs = gradio.HTML())

iface.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b389e86d0ce3743f2b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
